TRAVAUX PRATIQUES DE CLASSIFICATION NON-SUPERVISEE
==============================================

Ces travaux pratiques ont pour but d’appliquer et de manipuler les différents algorithmes de classification vus en cours de de classification non supervisée. Le cas d’application est l’identification de régions météorologiquement homogènes. Pour cela nous partirons du jeu de donnée Météonet, prétraité.

Enseignant : Thomas Rieutord (thomas.rieutord@meteo.fr)

Travail demandé :
-----------------
En suivant le sujet de TP qui vous a été donné, compléter les cellules vides de ce notebook, aux endroits marqués par ###### afin de répondre aux questions.
A la fin du TP, vous devriez pouvoir exécuter l'ensemble du programme en redémarrant et exécutant le notebook : `Noyau > Redémarrer & tout exécuter`.

In [ ]:
import os
import time
import numpy as np
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import coupdepouce as c2p

1- Position du problème
========================
Cette étape est en grande partie faite pour vous : il s'agit de définir, à l'aide d'une classification non-supervisée, des régions météorologiquement homogènes. Les données viennent du jeu de données MeteoNet et ont déjà été mises en forme. Les prédcteurs ont déjà été sélectionnés :
  * `Tn`: température minimale sur une journée et moyennée sur l'année
  * `Tx`: température maximale sur une journée et moyennée sur l'année
  * `FFmax`: force du vent à 10m maximale sur une journée et moyennée sur l'année
  * `RR`: cumul de précipitation sur l'année

Il ne reste plus qu'à charger les données en machine.

### Extraction
Ouvrir le fichier avec un éditeur de texte. Repérer les éléments séparateurs, la taille de l’en-tête et la nature des données (réel, entiers…).

En déduire la commande pour importer les données

In [ ]:
dataFile=os.path.join("..", "data", "obs_SE_2018.csv")

In [ ]:
df = ######
print("Retourne un ", type(df), " de taille ", df.shape)

In [ ]:
# Visualisation du dataframe dans le notebook
df

### Vérifications

Les prédicteurs sont-ils ceux attendus ?

In [ ]:
predictors = list(df.columns[:-2])
predictors

Combien y a-t-il de données manquantes ?

In [ ]:
# Nombre de données manquantes
nb_missingVal = ######
print("Nombre de données manquantes=", nb_missingVal)

### Normalisation

In [ ]:
# Normalisation
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X = ######

In [ ]:
N,p = X.shape
X.shape

2- Classification
==============
L'étape de classification nécessite le choix d'un nombre de clusters, bien que celui-ci soit souvent déduit de l'étape d'évaluation.
Cette valeur devra donc être réévaluée à l'issue de l'étape d'évaluation.

In [ ]:
K = ######

2-1 Classification hierarchique ascendante
--------------------------------------

In [ ]:
# Import du package scipy.cluster.hierarchy
from scipy.cluster import hierarchy as cha

Nous commencerons par une classification hiérarhique avec le critère de Ward et une distance euclidienne

In [ ]:
# Calcul de la matrice de connexion ('linkage matrix') -> cha.linkage"

linkageMatrix = ######

print("Retourne un ",type(linkageMatrix)," de taille ",np.shape(linkageMatrix))

In [ ]:
# Formation des groupes -> cha.fcluster

labels_cha = ######

**Attention :** nous souhaitons avoir des clusters numérotés entre 0 et K-1 et non entre 1 et K !


2-2 K-means
---------------------------------------------------
Il existe deux packages pour utiliser les K-means : `pyclustering` et `scikit-learn`

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
n_init=20

# Créer une instance de Kmeans
km_spy = ######

# Effectuer les calculs
######

# Récupérer les labels
labels_km_skl = ######

print("Kmeans (sklearn) retourne ", type(labels_km_skl), "de taille", len(labels_km_skl))
labels_km_skl

**Attention :** nous souhaitons avoir des clusters numérotés entre 0 et K-1 et non entre 1 et K !

In [ ]:
from pyclustering.cluster.kmeans import kmeans

In [ ]:
# On prend K points de l'echantillon que l'on perturbe
init_medoids = X[np.random.choice(N, K, replace=False), :] + 2*np.random.sample(p)-1

# Créer une instance de Kmeans
km_pyclust = ######

# Effectuer les calculs
######

# Récupérer les clusters
clusters_km_pyclust = ######
print("Kmeans (pyclustering) retourne ", type(clusters_km_pyclust), "de taille", len(clusters_km_pyclust))

# Transformation en labels (à la main)
labels_km_pyclust = np.zeros(N)
for k in range(K):
    labels_km_pyclust[clusters_km_pyclust[k]] = k

labels_km_pyclust

**Attention :** nous souhaitons avoir des clusters numérotés entre 0 et K-1 et non entre 1 et K !

2-3 K-medoids
--------------------------------------------------
Les K-medoids ne sont pas (encore) disponible dans `scikit-learn`.
Ils le sont dans une extension non-officielle et dans `pyclustering`.

In [ ]:
from pyclustering.cluster.kmedoids import kmedoids

In [ ]:
# Choix des points de départ (aléatoire)
init_medoids = np.random.choice(N, K, replace=False)
print(u"Choix des points de départ (aléatoire) : ", K, "points de l'échantillon. Indices :", init_medoids)

# Créer une instance de Kmedoids
pam = ######
# Effectuer les calculs
######
# Récupérer les clusters
clusters_pam = ######

print("Kmedoids (pyclustering) retourne ", type(clusters_pam), "de taille", len(clusters_pam))

# Transformation en labels
labels_pam = np.zeros(N)
for k in range(K):
    labels_pam[clusters_pam[k]] = k

labels_pam

**Attention :** nous souhaitons avoir des clusters numérotés entre 0 et K-1 et non entre 1 et K !

3- Évaluation
==============

3-1 Visualisation des classes
-----------------------------

Nous avons maintenant une classification des stations. Pour la commenter, il est important de pouvoir se représenter correctement ces classes. Nous cherchons ici à identifier des zones météorologiquement différentes selon la région. On va donc d’une part positionner les stations avec leur classe sur une carte et d'autre part regarder en quoi consistent les différences entre les clusters.

In [ ]:
labels = labels_cha
df['clusterlabel'] = labels

In [ ]:
# Choix de la colormap pour les clusters
import seaborn as sns
colmap = sns.color_palette("Paired", n_colors=K)

### Dendrogramme

In [ ]:
# Tracé du dendrogramme -> cha.dendrogram

plt.figure()
######
plt.ylabel("Distance cophénétique")
plt.xlabel("Stations")
plt.show(block=False)

### Pairplot

In [ ]:
# Tracé du diagramme par paire -> sns.pairplot
plt.figure()
######
plt.show(block=False)

### Sur une carte

In [ ]:
# Tracé sur une carte -> c2p.plot_clusters_on_map

######

3-2 Calcul de scores
--------------------------------------------------

In [ ]:
from sklearn import metrics

In [ ]:
# Calcul du score de silhouette -> sklearn.metrics.silhouette_samples
silhouette_values = ######
print("Retourne un", type(silhouette_values), " de taille ", silhouette_values.shape)

In [ ]:
# Calcul du score de silhouette -> sklearn.metrics.silhouette_score
sil_score = ######
print("Retourne un", type(sil_score), " de valeur ", sil_score)

In [ ]:
c2p.plot_silhouette(silhouette_values, labels, colormap=colmap)

Pour visualiser les clusters, réexécuter les cellules dédiées avec `labels=labels_km_spy` (par exemple).

### Choix du nombre de cluster optimal

In [ ]:
K_values = np.arange(2, 10)
sil_scores_cha = []
sil_scores_km = []

######
######

# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-
plt.figure()
plt.plot(K_values, sil_scores_cha, label="Hierarchique")
plt.plot(K_values, sil_scores_km, label="K-means")
plt.ylabel("Score silhouette")
plt.xlabel("Nombre de classes")
plt.grid()
plt.legend()
plt.show(block=False)
# -=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-

Deverrouillage de la correction
--------------------------------------------
Pour déverrouiller le notebook qui contient la correction, il vous faudra utiliser la commande `gpg correction.ipynb`.
Une phrase de passe vous sera demandée.
Il s'agit de la somme du score de silhouette obtenu avec une classification hiérarchique ascendante avec saut minimum et distance euclidienne pour K variant de 2 à 9, avec 6 décimales...

In [ ]:
######
######